In [7]:
import fiona
import geopandas as gpd
import numpy as np
import os
import qgrid

In [2]:
# os.path.expanduser("~") is the best way to get home directory, cross platform
path_to_glims_polygons = os.path.join(os.path.expanduser("~"), "Desktop", "projects", "glacier_segmentation", "glims_data","glims_polygons.shp")

In [3]:
df = gpd.read_file(path_to_glims_polygons)
df = df[['glac_id','src_date','area','db_area','geog_area','geometry']]

In [4]:
df_copy = df.copy()
df_copy = df_copy.to_crs({'proj':'cea'}) # Cylindrical equal-area, looks like this matches db_area from glims, cool.
df['my_area'] = df_copy['geometry'].area/(10**6)

In [21]:
Counter(df.geog_area)

Counter({'West Antarctica': 1,
         'Iceland': 251,
         'Randolph Glacier Inventory; Umbrella RC for merging the RGI into GLIMS': 172275,
         'Various (NSIDC)': 414,
         'Antarctic Peninsula': 5378,
         'East Central Greenland': 259,
         'Canada': 1146,
         'Svalbard, Jan Mayen, and Bouvet': 2652,
         'Various (GlobGlacier)': 40733,
         'Svalbard': 2587,
         'Asia': 130468,
         'Argentina': 19030,
         'Alaska': 3775,
         'Himalaya (India, Nepal, Bhutan)': 43149,
         'Chilean glaciers, between -18 and -56 degrees latitude': 109,
         'Khumbu Area': 3461,
         'Northern Cordillera': 17547,
         'Kerguelen Island': 337,
         'Prince William Sound': 208,
         'Siberian Mountains': 6060,
         'Russian and former Soviet Union glaciers': 91,
         'China': 48555,
         'Northeast Greenland': 51,
         'Russian Glaciers (exact boundaries of RC to be defined)': 1359,
         'Swiss Alps': 1908

In [5]:
df = df.sort_values(by = "my_area", ascending = False)
df.reset_index(drop = True)

,glac_id,src_date,area,db_area,geog_area,geometry,my_area
0,G263560E76894S,2001-10-01T00:00:00,165078.00,165079.00000,West Antarctica,"POLYGON Z ((-86.62274 -74.90813 0.00000, -86.6...",1.650792e+05
1,G343222E64409N,1999-01-01T00:00:00,0.00,8091.67000,Iceland,"MULTIPOLYGON Z (((-15.37719 64.66761 0.00000, ...",8.087962e+03
2,G344790E81287N,2003-07-01T00:00:00,7537.57,7537.75000,Randolph Glacier Inventory; Umbrella RC for me...,"POLYGON Z ((-17.27170 81.32675 0.00000, -17.27...",7.537750e+03
3,G298930E65309S,2000-02-21T00:00:00,0.00,7380.90000,Various (NSIDC),"POLYGON Z ((-61.70293 -65.53641 0.00000, -61.7...",7.380898e+03
4,G294015E69537S,2002-12-31T00:00:00,7018.37,7018.32000,Antarctic Peninsula,"POLYGON Z ((-65.06517 -69.48544 0.00000, -65.0...",7.018288e+03
...,...,...,...,...,...,...,...
548909,G213058E61576N,2007-07-16T00:00:00,0.00,0.00000,Alaska,"POLYGON Z ((-146.92007 61.58008 0.00000, -146....",-9.313226e-16
548910,G226291E58952N,2001-08-15T00:00:00,0.00,3.92986,Various (NSIDC),"POLYGON Z ((-133.68569 58.96223 0.00000, -133....",-3.943725e+00
548911,G214152E61305N,2007-07-16T00:00:00,0.00,42.69420,Alaska,"POLYGON Z ((-145.90074 61.29870 0.00000, -145....",-4.269424e+01
548912,G211615E61175N,2007-07-16T00:00:00,0.00,226.41100,Alaska,"POLYGON Z ((-148.25825 61.16509 0.00000, -148....",-2.719483e+02


In [13]:
from collections import Counter

In [16]:
df.shape

(548914, 7)

In [15]:
len(Counter(df.glac_id))

324929

In [10]:
qgrid.show_grid(df.head(100), grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})

OverflowError: Maximum recursion level reached

In [8]:
df_2 = df.iloc[0:100,:]

In [78]:
scale_fact = 1.1
temp = df.envelope.scale(xfact=scale_fact, yfact=scale_fact).squeeze().exterior

In [79]:
temp_list = [list(zip(*np.asarray(t.coords.xy))) for t in temp]

In [81]:
df['bboxes'] = temp_list

In [83]:
path_to_master_glims_df = os.path.join(os.path.expanduser("~"), "Desktop", "projects","glacier_segmentation","glims_data","master_glims_df.csv")
df.to_csv(path_to_master_glims_df)